<a href="https://colab.research.google.com/github/Minjk121/Minjk121/blob/main/mjk_pipline_iris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
%%bash
pip install imbalanced-learn
pip install pennylane
pip install pennylane_cirq

In [ ]:
import pandas
import torch
import pennylane as qml
from pennylane import numpy as np
import torch.nn
import matplotlib.pyplot as plt
from pennylane.templates import AmplitudeEmbedding
import random
import os
import copy
from pennylane_cirq import ops as cirq_ops
from sklearn.model_selection import KFold
from sklearn import preprocessing
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTENC
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

In [ ]:
dev = qml.device("cirq.mixedsimulator", wires = 2)
projector = torch.zeros((2,2))
projector[0,0] = 1

encoding = 'DenseAngle'
channel = 'amplitude damp'
p = 0.3
split_proportion = 0.8

In [ ]:
def load_data(split_proportion=0.8):
  df = pandas.read_csv('https://raw.githubusercontent.com/mcmahon-lab/qml/main/Robust/FIG_10/IRIS.csv?token=AJGBMBPCPB7OZIXRDIQ7OR3BX6QBI')
  # use 2 target values
  df = df.loc[df["species"] != "Iris-versicolor"]
  df.species[df.species == "Iris-setosa"] = 0
  df.species[df.species == "Iris-virginica"] = 1
  data = df.filter(items=['sepal_length', 'sepal_width','petal_length','petal_width'])
  labels = df.filter(items=['species'])
  length = len(df)
  df = df.sample(frac = 1).astype(float)
  cut_off = (int)(length*split_proportion)
  train = torch.tensor(df[:cut_off].drop(columns=["species"]).values, dtype=torch.float64)
  val = torch.tensor(df[cut_off:].drop(columns=["species"]).values, dtype=torch.float64)
  train_target = torch.tensor(df.species[:cut_off].values, dtype=torch.float64)
  val_target = torch.tensor(df.species[cut_off:].values, dtype=torch.float64)
  # train_data = torch.tensor(train_data.values, dtype=torch.float64)
  # test_data = torch.tensor(test_data.values, dtype=torch.float64)
  # train_labels = torch.tensor(train_labels.values, dtype=torch.float64)
  # test_labels = torch.tensor(train_labels.values, dtype=torch.float64)
  # return train_data, test_data, train_labels, test_labels
  return torch.tensor(train), torch.tensor(train_target), val, val_target
load_data(0.8)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


(tensor([[7.2000, 3.2000, 6.0000, 1.8000],
         [5.1000, 3.5000, 1.4000, 0.2000],
         [7.9000, 3.8000, 6.4000, 2.0000],
         [6.4000, 2.8000, 5.6000, 2.1000],
         [5.0000, 3.4000, 1.6000, 0.4000],
         [4.6000, 3.6000, 1.0000, 0.2000],
         [4.4000, 2.9000, 1.4000, 0.2000],
         [5.1000, 3.8000, 1.6000, 0.2000],
         [5.5000, 3.5000, 1.3000, 0.2000],
         [6.1000, 2.6000, 5.6000, 1.4000],
         [6.4000, 2.7000, 5.3000, 1.9000],
         [5.0000, 3.5000, 1.3000, 0.3000],
         [5.8000, 2.7000, 5.1000, 1.9000],
         [6.0000, 2.2000, 5.0000, 1.5000],
         [5.1000, 3.5000, 1.4000, 0.3000],
         [7.7000, 2.6000, 6.9000, 2.3000],
         [4.3000, 3.0000, 1.1000, 0.1000],
         [6.3000, 2.9000, 5.6000, 1.8000],
         [4.6000, 3.2000, 1.4000, 0.2000],
         [4.8000, 3.4000, 1.6000, 0.2000],
         [6.1000, 3.0000, 4.9000, 1.8000],
         [5.9000, 3.0000, 5.1000, 1.8000],
         [5.4000, 3.9000, 1.3000, 0.4000],
         [5

In [ ]:
dev = qml.device('cirq.mixedsimulator', wires=2)
@qml.qnode(dev, interface = "torch")
def circuit(inputs, params_circ, params_emb):
  global encoding, channel, noise, p
  if encoding == 'amplitude':
      AmplitudeEmbedding(features = inputs, wires = range(2), normalize = True)
  elif encoding == 'DenseAngle':
      qml.RY(2*params_emb[0]*inputs[0], wires= 0)
      qml.PhaseShift(2*params_emb[1]*inputs[1], wires = 0)
      qml.RY(2*params_emb[2]*inputs[2], wires= 1)
      qml.PhaseShift(2*params_emb[3]*inputs[3], wires = 1)
  elif encoding == 'SuperDenseAngle':
      qml.RY(2*params_emb[0]*inputs[0] + 2*params_emb[1]*inputs[1], wires = 0)
      qml.RY(2*params_emb[2]*inputs[2] + 2*params_emb[3]*inputs[3], wires = 1)
  ansatz(params_circ)
  if noise:
    if channel == 'pauli':
      #TODO
      pass
    elif channel == 'depolarizing':
      cirq_ops.Depolarize(p, wires = 0)
      cirq_ops.Depolarize(p, wires = 1)
    elif channel == 'amplitude damp':
      cirq_ops.AmplitudeDamp(p, wires = 0)
      cirq_ops.AmplitudeDamp(p, wires = 1)
  return qml.expval(qml.Hermitian(projector, wires = 0))


In [ ]:
def ansatz(params_circ):
  qml.Rot(2*params_circ[0], 2*params_circ[1], 2*params_circ[2], wires = 0)
  qml.Rot(2*params_circ[3], 2*params_circ[4], 2*params_circ[5], wires = 1)
  qml.CNOT(wires=[0, 1])
  qml.Rot(2*params_circ[6], 2*params_circ[7], 2*params_circ[8], wires = 0)
  qml.Rot(2*params_circ[9], 2*params_circ[10], 2*params_circ[11], wires = 1)

In [ ]:
def freeze_layer(QNN, n):
    """
    Freeze the nth layer of QNN.
    n = 1: frozen layer is the unitary layer.
    n = 0: frozen layer is the embedding layer.
    """
    for idx, param in enumerate(QNN.parameters()):
        if idx == n:
            param.requires_grad = False
        else:
            param.requires_grad = True
    return QNN

In [ ]:
global noise, encoding, channel, batch_size, num_of_runs
params_shapes = {"params_circ": 12, "params_emb": 4}
cost_funcs = [torch.nn.BCELoss(),torch.nn.MSELoss()]
num_folds = [4, 8, 12]
best_param = None
best_score = 0
best_model = None
lrs = [1, 0.1, 0.01, 0.001]
encode_lrs = [1, 0.1, 0.01, 0.001]
itr = 1
encode_itrs = 1
batch_sizes = [10, 12, 14]
num_of_runs = 1
train, train_target, val, val_target  = load_data(0.8)
shuffler = torch.randperm(len(train))
train = train[shuffler]
train_target = train_target[shuffler]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [ ]:
def accuracy(output, label):
  accum = 0
  length = len(output)
  for i in range(length):
    if label[i] - output[i] >= -0.5 and label[i] - output[i] <= 0.5:
      accum += 1
  return accum/length

In [ ]:
for batch_size in batch_sizes:
  for cost in cost_funcs:
    for lr in lrs:
        for encode_lr in encode_lrs:
            for num_fold in num_folds:
                kf = KFold(n_splits=num_fold)
                score = 0
                for train_index, test_index in kf.split(train):
                  train_data, test_data = train[train_index], train[test_index]
                  train_label, test_label = train_target[train_index], train_target[test_index]
                  #train circuit parameters
                  noise = False
                  QNN = qml.qnn.TorchLayer(circuit, params_shapes)
                  QNN = freeze_layer(QNN, 1)
                  opt_circuit = torch.optim.Adam(QNN.parameters(), lr = lr)
                  lowest_cost_noise = 1e10
                  lowest_QNN_state_noise = None
                  QNN_state = {"params_circ": torch.FloatTensor(np.random.uniform(0,2*np.pi,12)), "params_emb": torch.FloatTensor(np.random.uniform(0,2*np.pi,4))}
                  QNN.load_state_dict(QNN_state)
                  lowest_cost = 1e10
                  lowest_QNN_state = None
                  for iter in range(itr):
                    total_tr_cost = 0
                    #stochastic gradient decent
                    for i in range(0, len(train_data), batch_size):
                        opt_circuit.zero_grad()
                        end_index = i+batch_size
                        train_pts = train_data[i:end_index]
                        output = QNN(train_pts)
                        train_cost = cost(output,train_label[i:end_index])
                        total_tr_cost += train_cost.data.numpy()
                        train_cost.backward()
                        opt_circuit.step()
                    if lowest_cost > total_tr_cost:
                      lowest_cost = total_tr_cost
                      lowest_QNN_state = copy.deepcopy(QNN.state_dict())
                    print((str)(run)+" runs, "+(str)(iter)+" iterations, cost is "+(str)(total_tr_cost)+"，QNN state: "+(str)(QNN.state_dict()))
                  print("lowest cost: "+(str)(lowest_cost)+", lowest_QNN_state: "+(str)(lowest_QNN_state ))
                  noise = True
                  QNN_noise = qml.qnn.TorchLayer(circuit, params_shapes)
                  QNN_noise.load_state_dict(lowest_QNN_state)
                  QNN_noise = freeze_layer(QNN_noise, 0)
                  opt_embeddings = torch.optim.Adam(QNN_noise.parameters(), lr = encode_lr)
                  for iter in range(encode_itrs):
                    total_tr_cost_noise = 0
                    #stochastic gradient decent
                    for i in range(0, len(train_data), batch_size):
                      # print(QNN_noise.state_dict())
                      opt_embeddings.zero_grad()
                      end_index = i+batch_size
                      train_pts = train_data[i:end_index]
                      output_noise = QNN_noise(train_pts)
                      train_cost_noise = cost(output_noise,train_label[i:end_index])
                      total_tr_cost_noise += train_cost_noise.data.numpy()
                      train_cost_noise.backward()
                      opt_embeddings.step()
                    if lowest_cost_noise > total_tr_cost_noise:
                      lowest_cost_noise = total_tr_cost_noise
                      lowest_QNN_state_noise= copy.deepcopy(QNN_noise.state_dict())
                  #   print((str)(run)+" runs, "+(str)(iter)+" iterations, cost noise is "+(str)(total_tr_cost_noise)+"，QNN state noise: "+(str)(QNN_noise.state_dict()))
                  # print("so far lowest cost is "+(str)(lowest_cost_noise))
                  noise = True
                  QNN_test = qml.qnn.TorchLayer(circuit, params_shapes)
                  QNN_test.load_state_dict(lowest_QNN_state_noise)
                  QNN_test.state_dict().requires_grad = False
                  output_test = QNN_test(test_data)
                  accur = accuracy(output_test, test_label)
                  score += accur
                score = score/num_fold
                if score > best_score:
                  best_param, best_score, best_model = (batch_size, cost, lr, encode_lr, num_fold), score, lowest_QNN_state_noise
print("best param: "+(str)(best_param))
print("best score: "+(str)(best_score))
print("best qnn: "+(str)(best_model))
file = open('results1.txt','w')
file.write("best param: "+(str)(best_param)+'\n')
file.write("best score: "+(str)(best_score)+'\n')
file.write("best qnn: "+(str)(best_model)+'\n')
file.close()
